In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import Conv2D, MaxPool2D, Flatten
from keras.layers import Dense, Input, Dropout, BatchNormalization, UpSampling2D
from keras.utils import to_categorical
from keras.models import Sequential
from keras.models import *
import cv2
import os
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
train1 = 'C:\\Users\\JYOTHI PRASANNA\\Desktop\\dog_cat_new\\train1\\'
train2 = 'C:\\Users\\JYOTHI PRASANNA\\Desktop\\dog_cat_new\\train2\\'
train3 = 'C:\\Users\\JYOTHI PRASANNA\\Desktop\\dog_cat_new\\train3\\'
train4 = 'C:\\Users\\JYOTHI PRASANNA\\Desktop\\dog_cat_new\\train4\\'
train5 = 'C:\\Users\\JYOTHI PRASANNA\\Desktop\\dog_cat_new\\train5\\'

train_4 = 'C:\\Users\\JYOTHI PRASANNA\\Desktop\\dog_cat_new\\trai_n\\'
train_5 = 'C:\\Users\\JYOTHI PRASANNA\\Desktop\\dog_cat_new\\train\\'
test = 'C:\\Users\\JYOTHI PRASANNA\\Desktop\\dog_cat_new\\test\\'

In [3]:
filenames = sorted(os.listdir(train1))
p4=0
d4=[]
for img_name in filenames:
    img = plt.imread(train1 + img_name)
    img  = np.resize(img, (28,28))
    if p4==0:
      imgs4=(img)
      p4=1
    else:
      imgs4 = np.append(imgs4, img, axis=0)
    res = img_name[:3]
    d4.append(res)
print(d4)
    
print(imgs4.shape)
        
img_data = np.array(imgs4)
img_data = img_data.astype('float32')
img_data = img_data/255
img_data.shape

['cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor'

(5600, 28)

In [4]:
imgs4 = imgs4.reshape(-1,28,28,1)
#test_images = test_images.reshape(-1,256,256,1)
imgs4.shape#,test_images.shape

(200, 28, 28, 1)

In [5]:
imgs4 = imgs4 / np.max(imgs4)
#test_images = test_images / np.max(test_images)

In [6]:
train_X,valid_X,train_ground,valid_ground = train_test_split(imgs4,imgs4,test_size=0.2,random_state=13)

In [10]:
input = Input(shape = (28,28,1))
def encoder(input):
    #encoder
    #input = 28 x 28 x 1 (wide and thin)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input) #28 x 28 x 32
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPool2D(pool_size=(2, 2))(conv1) #14 x 14 x 32
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1) #14 x 14 x 64
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPool2D(pool_size=(2, 2))(conv2) #7 x 7 x 64
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2) #7 x 7 x 128 (small and thick)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)

    return conv3

def decoder(conv3):    
    #decoder
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv3) #7 x 7 x 64
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    up1 = UpSampling2D((2,2))(conv6) #14 x 14 x 64
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up1) # 14 x 14 x 32
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    up2 = UpSampling2D((2,2))(conv7) # 28 x 28 x 32
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(up2) # 28 x 28 x 1
    return decoded


autoencoder = Model(input, decoder(encoder(input)))
autoencoder.compile(loss='mean_squared_error', optimizer = 'rmsprop',metrics =['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
autoencoder_train = autoencoder.fit(train_X, train_ground, batch_size=32,epochs=5,verbose=1,validation_data=(valid_X, valid_ground))

Instructions for updating:
Use tf.cast instead.
Train on 160 samples, validate on 40 samples
Epoch 1/5
160/160 [==============================] - 3s 21ms/step - loss: 0.1006 - acc: 0.0114 - val_loss: 0.0840 - val_acc: 0.0411
Epoch 2/5
160/160 [==============================] - 2s 12ms/step - loss: 0.0360 - acc: 0.0135 - val_loss: 0.0336 - val_acc: 0.0421
Epoch 3/5
160/160 [==============================] - 2s 13ms/step - loss: 0.0261 - acc: 0.0137 - val_loss: 0.0282 - val_acc: 0.0421
Epoch 4/5
160/160 [==============================] - 2s 14ms/step - loss: 0.0375 - acc: 0.0134 - val_loss: 0.0366 - val_acc: 0.0419
Epoch 5/5
160/160 [==============================] - 2s 14ms/step - loss: 0.0244 - acc: 0.0137 - val_loss: 0.0410 - val_acc: 0.0421


In [12]:
autoencoder.evaluate(valid_X, valid_ground,verbose=0)

[0.040956272184848784, 0.04212372414767742]

In [13]:
filenames = sorted(os.listdir(train1))
p4=0
d4=[]
for img_name in filenames:
    img = plt.imread(train1 + img_name)
    img  = np.resize(img, (28,28))
    if p4==0:
      imgs4=(img)
      p4=1
    else:
      imgs4 = np.append(imgs4, img, axis=0)
    res = img_name[:3]
    d4.append(res)
print(d4)
    
print(imgs4.shape)
        
img_data = np.array(imgs4)
img_data = img_data.astype('float32')
img_data = img_data/255
img_data.shape

['cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor'

(5600, 28)

In [14]:
imgs4 = np.reshape(imgs4, [ 200, 28, 28])
imgs = np.reshape(imgs4, [ 200, 28, 28])
train_images, test_images, train_labels, test_labels = train_test_split(imgs4, d4, test_size=0.33, random_state=42)
print('Training data shape : ', train_images.shape, len(train_labels))
print('Testing data shape : ', test_images.shape, len(test_labels))
classes = np.unique(train_labels)
#classes=np.append(classes,0)
nClasses = len(classes)

print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Training data shape :  (134, 28, 28) 134
Testing data shape :  (66, 28, 28) 66
Total number of outputs :  4
Output classes :  ['cat' 'dog' 'hor' 'hum']


In [15]:
train_labels=[0 if x=='cat' else 1 for x in train_labels]
print(train_labels)
len(train_labels)

[0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1]


134

In [16]:
test_labels=[0 if x=='cat' else 1 for x in test_labels]
print(test_labels)
len(test_labels)

[0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1]


66

In [17]:
nRows,nCols = train_images.shape[1:]
nDims = nRows
print(nCols)
train_data = train_images.reshape(train_images.shape[0], nRows, nCols, 1)
test_data = test_images.reshape(test_images.shape[0], nRows, nCols, 1)
input_shape = (nRows, nCols, 1)
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')
train_data /= 255
test_data /= 255
print(len(train_labels))
print(len(test_labels))
train_labels_one_hot = to_categorical(train_labels)
test_labels_one_hot = to_categorical(test_labels)
print('Original label : ', train_labels[56])
print('After conversion to categorical ( one-hot ) : ', train_labels_one_hot[56])
print('Original label : ', test_labels[56])
print('After conversion to categorical ( one-hot ) : ', test_labels_one_hot[56])

28
134
66
Original label :  1
After conversion to categorical ( one-hot ) :  [0. 1.]
Original label :  1
After conversion to categorical ( one-hot ) :  [0. 1.]


In [18]:
def fc(enco):
    flat = Flatten()(enco)
    den = Dense(128, activation='relu')(flat)
    out = Dense(2, activation='softmax')(den)
    return out

encode = encoder(input)
full_model = Model(input,fc(encode))

In [19]:
for l1,l2 in zip(full_model.layers[:14],autoencoder.layers[0:14]):
    l1.set_weights(l2.get_weights())

In [20]:
for layer in full_model.layers[0:14]:
    layer.trainable = False

In [21]:
full_model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [22]:
classify_train = full_model.fit(train_data, train_labels_one_hot, batch_size=32,epochs=10,verbose=1,validation_data=(test_data, test_labels_one_hot))

Train on 134 samples, validate on 66 samples
Epoch 1/10
134/134 [==============================] - 1s 8ms/step - loss: 1.5086 - acc: 0.7090 - val_loss: 1.4135 - val_acc: 0.7727
Epoch 2/10
134/134 [==============================] - 0s 4ms/step - loss: 1.2726 - acc: 0.8060 - val_loss: 2.3226 - val_acc: 0.7727
Epoch 3/10
134/134 [==============================] - 1s 4ms/step - loss: 1.4541 - acc: 0.8507 - val_loss: 1.6917 - val_acc: 0.8485
Epoch 4/10
134/134 [==============================] - 1s 4ms/step - loss: 0.7979 - acc: 0.8955 - val_loss: 1.9342 - val_acc: 0.6667
Epoch 5/10
134/134 [==============================] - 1s 4ms/step - loss: 1.2454 - acc: 0.8507 - val_loss: 1.8552 - val_acc: 0.6667
Epoch 6/10
134/134 [==============================] - 1s 4ms/step - loss: 1.3050 - acc: 0.8657 - val_loss: 1.6529 - val_acc: 0.8485
Epoch 7/10
134/134 [==============================] - 1s 4ms/step - loss: 1.5395 - acc: 0.7985 - val_loss: 1.6183 - val_acc: 0.8182
Epoch 8/10
134/134 [===========

In [23]:
full_model.evaluate(test_data, test_labels_one_hot)

66/66 [==============================] - 0s 2ms/step


[1.6621364029971035, 0.7575757575757576]

In [24]:
filenames = sorted(os.listdir(train2))
p4=0
d41=[]
for img_name in filenames:
    img = plt.imread(train2 + img_name)
    img  = np.resize(img, (28,28))
    if p4==0:
      imgs41=(img)
      p4=1
    else:
      imgs41 = np.append(imgs41, img, axis=0)
    res = img_name[:3]
    d41.append(res)
print(d41)
    
print(imgs41.shape)
        
img_data = np.array(imgs41)
img_data = img_data.astype('float32')
img_data = img_data/255
img_data.shape

['cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor'

(5572, 28)

In [25]:
imgs441 = imgs41.reshape(-1,28,28,1)
print(imgs441.shape)
imgs441 = imgs441 / np.max(imgs441)

(199, 28, 28, 1)


In [26]:
train_X1,valid_X1,train_ground1,valid_ground1 = train_test_split(imgs441,imgs441,test_size=0.2,random_state=13)

In [27]:
def encoder1(input):
    #encoder
    #input = 28 x 28 x 1 (wide and thin)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input) #28 x 28 x 32
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPool2D(pool_size=(2, 2))(conv1) #14 x 14 x 32
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1) #14 x 14 x 64
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPool2D(pool_size=(2, 2))(conv2) #7 x 7 x 64
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2) #7 x 7 x 128 (small and thick)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)

    return conv3

def decoder1(conv3):    
    #decoder
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv3) #7 x 7 x 64
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    up1 = UpSampling2D((2,2))(conv6) #14 x 14 x 64
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up1) # 14 x 14 x 32
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    up2 = UpSampling2D((2,2))(conv7) # 28 x 28 x 32
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(up2) # 28 x 28 x 1
    return decoded


autoencoder1 = Model(input, decoder1(encoder1(input)))
autoencoder1.compile(loss='mean_squared_error', optimizer = 'rmsprop',metrics =['accuracy'])

In [28]:
autoencoder_train1 = autoencoder1.fit(train_X1, train_ground1, batch_size=32,epochs=5,verbose=1,validation_data=(valid_X1, valid_ground1))


Train on 159 samples, validate on 40 samples
Epoch 1/5
159/159 [==============================] - 4s 27ms/step - loss: 0.1064 - acc: 0.0140 - val_loss: 0.0820 - val_acc: 0.0186
Epoch 2/5
159/159 [==============================] - 3s 16ms/step - loss: 0.0429 - acc: 0.0176 - val_loss: 0.0330 - val_acc: 0.0191
Epoch 3/5
159/159 [==============================] - 2s 15ms/step - loss: 0.0255 - acc: 0.0178 - val_loss: 0.0333 - val_acc: 0.0193
Epoch 4/5
159/159 [==============================] - 3s 17ms/step - loss: 0.0282 - acc: 0.0176 - val_loss: 0.0317 - val_acc: 0.0195
Epoch 5/5
159/159 [==============================] - 3s 16ms/step - loss: 0.0262 - acc: 0.0177 - val_loss: 0.0375 - val_acc: 0.0194


In [29]:
imgs41 = np.reshape(imgs41, [ 199, 28, 28])
imgs1 = np.reshape(imgs41, [ 199, 28, 28])
train_images1, test_images1, train_labels1, test_labels1 = train_test_split(imgs41, d41, test_size=0.33, random_state=42)
print('Training data shape : ', train_images1.shape, len(train_labels1))
print('Testing data shape : ', test_images1.shape, len(test_labels1))
classes = np.unique(train_labels1)
#classes=np.append(classes,0)
nClasses = len(classes)

print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Training data shape :  (133, 28, 28) 133
Testing data shape :  (66, 28, 28) 66
Total number of outputs :  4
Output classes :  ['cat' 'dog' 'hor' 'hum']


In [30]:
train_labels1=[0 if x=='dog' else 1 for x in train_labels1]
print(train_labels1)
len(train_labels1)

[0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0]


133

In [31]:
test_labels1=[0 if x=='dog' else 1 for x in test_labels1]
print(test_labels1)
len(test_labels1)

[0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0]


66

In [32]:
nRows,nCols = train_images1.shape[1:]
nDims = nRows
print(nCols)
train_data1 = train_images1.reshape(train_images1.shape[0], nRows, nCols, 1)
test_data1 = test_images1.reshape(test_images1.shape[0], nRows, nCols, 1)
input_shape = (nRows, nCols, 1)
train_data1 = train_data1.astype('float32')
test_data1 = test_data1.astype('float32')
train_data1 /= 255
test_data1 /= 255
print(len(train_labels1))
print(len(test_labels1))
train_labels_one_hot1 = to_categorical(train_labels1)
test_labels_one_hot1 = to_categorical(test_labels1)
print('Original label : ', train_labels1[56])
print('After conversion to categorical ( one-hot ) : ', train_labels_one_hot1[56])
print('Original label : ', test_labels1[56])
print('After conversion to categorical ( one-hot ) : ', test_labels_one_hot1[56])

28
133
66
Original label :  1
After conversion to categorical ( one-hot ) :  [0. 1.]
Original label :  0
After conversion to categorical ( one-hot ) :  [1. 0.]


In [33]:
def fc(enco):
    flat = Flatten()(enco)
    den = Dense(128, activation='relu')(flat)
    out = Dense(2, activation='softmax')(den)
    return out

encode1 = encoder1(input)
full_model1 = Model(input,fc(encode1))

In [34]:
for l1,l2 in zip(full_model1.layers[:14],autoencoder1.layers[0:14]):
    l1.set_weights(l2.get_weights())

In [35]:
for layer in full_model1.layers[0:14]:
    layer.trainable = False

In [36]:
full_model1.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [37]:
classify_train1 = full_model1.fit(train_data1, train_labels_one_hot1, batch_size=32,epochs=10,verbose=1,validation_data=(test_data1, test_labels_one_hot1))

Train on 133 samples, validate on 66 samples
Epoch 1/10
133/133 [==============================] - 1s 11ms/step - loss: 1.7606 - acc: 0.7444 - val_loss: 3.0464 - val_acc: 0.8030
Epoch 2/10
133/133 [==============================] - 0s 4ms/step - loss: 1.6868 - acc: 0.8571 - val_loss: 1.8852 - val_acc: 0.8333
Epoch 3/10
133/133 [==============================] - 0s 4ms/step - loss: 1.1927 - acc: 0.8496 - val_loss: 2.7950 - val_acc: 0.7727
Epoch 4/10
133/133 [==============================] - 0s 4ms/step - loss: 1.1490 - acc: 0.8571 - val_loss: 2.6684 - val_acc: 0.7727
Epoch 5/10
133/133 [==============================] - 1s 4ms/step - loss: 0.9418 - acc: 0.8571 - val_loss: 2.4836 - val_acc: 0.7879
Epoch 6/10
133/133 [==============================] - 0s 4ms/step - loss: 0.9444 - acc: 0.9023 - val_loss: 2.8798 - val_acc: 0.7576
Epoch 7/10
133/133 [==============================] - 1s 4ms/step - loss: 1.1438 - acc: 0.8571 - val_loss: 2.8485 - val_acc: 0.7576
Epoch 8/10
133/133 [==========

In [38]:
full_model1.evaluate(test_data1, test_labels_one_hot1)

66/66 [==============================] - 0s 2ms/step


[2.7449387293873566, 0.7424242424242424]

In [39]:
filenames = sorted(os.listdir(train3))
p4=0
d42=[]
for img_name in filenames:
    img = plt.imread(train3 + img_name)
    img  = np.resize(img, (28,28))
    if p4==0:
      imgs42=(img)
      p4=1
    else:
      imgs42 = np.append(imgs42, img, axis=0)
    res = img_name[:3]
    d42.append(res)
print(d42)
    
print(imgs42.shape)
        
img_data = np.array(imgs42)
img_data = img_data.astype('float32')
img_data = img_data/255
img_data.shape

['cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor'

(5600, 28)

In [40]:
imgs442 = imgs42.reshape(-1,28,28,1)
print(imgs442.shape)
imgs442 = imgs442 / np.max(imgs442)

(200, 28, 28, 1)


In [41]:
train_X2,valid_X2,train_ground2,valid_ground2 = train_test_split(imgs442,imgs442,test_size=0.2,random_state=13)

In [42]:
def encoder2(input):
    #encoder
    #input = 28 x 28 x 1 (wide and thin)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input) #28 x 28 x 32
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPool2D(pool_size=(2, 2))(conv1) #14 x 14 x 32
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1) #14 x 14 x 64
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPool2D(pool_size=(2, 2))(conv2) #7 x 7 x 64
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2) #7 x 7 x 128 (small and thick)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)

    return conv3

def decoder2(conv3):    
    #decoder
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv3) #7 x 7 x 64
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    up1 = UpSampling2D((2,2))(conv6) #14 x 14 x 64
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up1) # 14 x 14 x 32
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    up2 = UpSampling2D((2,2))(conv7) # 28 x 28 x 32
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(up2) # 28 x 28 x 1
    return decoded


autoencoder2 = Model(input, decoder2(encoder2(input)))
autoencoder2.compile(loss='mean_squared_error', optimizer = 'rmsprop',metrics =['accuracy'])

In [43]:
autoencoder_train2 = autoencoder2.fit(train_X2, train_ground2, batch_size=32,epochs=5,verbose=1,validation_data=(valid_X2, valid_ground2))


Train on 160 samples, validate on 40 samples
Epoch 1/5
160/160 [==============================] - 4s 23ms/step - loss: 0.1757 - acc: 0.0085 - val_loss: 0.1283 - val_acc: 8.6097e-04
Epoch 2/5
160/160 [==============================] - 2s 12ms/step - loss: 0.0529 - acc: 0.0155 - val_loss: 0.0301 - val_acc: 0.0016
Epoch 3/5
160/160 [==============================] - 2s 12ms/step - loss: 0.0465 - acc: 0.0153 - val_loss: 0.0351 - val_acc: 0.0014
Epoch 4/5
160/160 [==============================] - 2s 14ms/step - loss: 0.0281 - acc: 0.0164 - val_loss: 0.0166 - val_acc: 0.0016
Epoch 5/5
160/160 [==============================] - 2s 13ms/step - loss: 0.0207 - acc: 0.0163 - val_loss: 0.0094 - val_acc: 0.0016


In [44]:
imgs42 = np.reshape(imgs42, [ 200, 28, 28])
imgs1 = np.reshape(imgs42, [ 200, 28, 28])
train_images2, test_images2, train_labels2, test_labels2 = train_test_split(imgs42, d42, test_size=0.33, random_state=42)
print('Training data shape : ', train_images2.shape, len(train_labels2))
print('Testing data shape : ', test_images2.shape, len(test_labels2))
classes = np.unique(train_labels2)
#classes=np.append(classes,0)
nClasses = len(classes)

print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Training data shape :  (134, 28, 28) 134
Testing data shape :  (66, 28, 28) 66
Total number of outputs :  4
Output classes :  ['cat' 'dog' 'hor' 'hum']


In [45]:
train_labels2=[0 if x=='hor' else 1 for x in train_labels2]
print(train_labels2)
len(train_labels2)

[1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0]


134

In [46]:
test_labels2=[0 if x=='hor' else 1 for x in test_labels2]
print(test_labels2)
len(test_labels2)

[0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0]


66

In [47]:
nRows,nCols = train_images2.shape[1:]
nDims = nRows
print(nCols)
train_data2 = train_images2.reshape(train_images2.shape[0], nRows, nCols, 1)
test_data2 = test_images2.reshape(test_images2.shape[0], nRows, nCols, 1)
input_shape = (nRows, nCols, 1)
train_data2 = train_data2.astype('float32')
test_data2 = test_data2.astype('float32')
train_data2 /= 255
test_data2 /= 255
print(len(train_labels2))
print(len(test_labels2))
train_labels_one_hot2 = to_categorical(train_labels2)
test_labels_one_hot2 = to_categorical(test_labels2)
print('Original label : ', train_labels2[56])
print('After conversion to categorical ( one-hot ) : ', train_labels_one_hot2[56])
print('Original label : ', test_labels2[56])
print('After conversion to categorical ( one-hot ) : ', test_labels_one_hot2[56])

28
134
66
Original label :  0
After conversion to categorical ( one-hot ) :  [1. 0.]
Original label :  0
After conversion to categorical ( one-hot ) :  [1. 0.]


In [48]:
def fc1(enco):
    flat = Flatten()(enco)
    den = Dense(128, activation='relu')(flat)
    out = Dense(2, activation='softmax')(den)
    return out

encode2 = encoder2(input)
full_model2 = Model(input,fc1(encode2))

In [49]:
for l1,l2 in zip(full_model2.layers[:14],autoencoder2.layers[0:14]):
    l1.set_weights(l2.get_weights())

In [50]:
for layer in full_model2.layers[0:14]:
    layer.trainable = False

In [51]:
full_model2.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [52]:
classify_train1 = full_model2.fit(train_data2, train_labels_one_hot2, batch_size=32,epochs=10,verbose=1,validation_data=(test_data2, test_labels_one_hot2))

Train on 134 samples, validate on 66 samples
Epoch 1/10
134/134 [==============================] - 2s 13ms/step - loss: 2.1194 - acc: 0.6343 - val_loss: 2.2622 - val_acc: 0.8485
Epoch 2/10
134/134 [==============================] - 1s 4ms/step - loss: 2.5904 - acc: 0.8358 - val_loss: 2.2770 - val_acc: 0.8485
Epoch 3/10
134/134 [==============================] - 1s 4ms/step - loss: 2.5989 - acc: 0.8358 - val_loss: 1.3560 - val_acc: 0.8636
Epoch 4/10
134/134 [==============================] - 1s 4ms/step - loss: 2.5019 - acc: 0.8433 - val_loss: 0.3266 - val_acc: 0.8636
Epoch 5/10
134/134 [==============================] - 1s 4ms/step - loss: 1.3872 - acc: 0.6940 - val_loss: 0.4471 - val_acc: 0.8788
Epoch 6/10
134/134 [==============================] - 1s 4ms/step - loss: 1.6552 - acc: 0.8433 - val_loss: 3.5356 - val_acc: 0.4697
Epoch 7/10
134/134 [==============================] - 1s 4ms/step - loss: 1.2479 - acc: 0.6269 - val_loss: 0.3467 - val_acc: 0.8182
Epoch 8/10
134/134 [==========

In [53]:
full_model2.evaluate(test_data2, test_labels_one_hot2)

66/66 [==============================] - 0s 2ms/step


[0.3263032470237125, 0.8636363636363636]

In [54]:
filenames = sorted(os.listdir(train4))
p4=0
d43=[]
for img_name in filenames:
    img = plt.imread(train4 + img_name)
    img  = np.resize(img, (28,28))
    if p4==0:
      imgs43=(img)
      p4=1
    else:
      imgs43 = np.append(imgs43, img, axis=0)
    res = img_name[:3]
    d43.append(res)
print(d43)
    
print(imgs43.shape)
        
img_data = np.array(imgs43)
img_data = img_data.astype('float32')
img_data = img_data/255
img_data.shape

['cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum', 'hum'

(5600, 28)

In [55]:
imgs443 = imgs43.reshape(-1,28,28,1)
print(imgs442.shape)
imgs443 = imgs443 / np.max(imgs443)

(200, 28, 28, 1)


In [56]:
train_X3,valid_X3,train_ground3,valid_ground3 = train_test_split(imgs443,imgs443,test_size=0.2,random_state=13)

In [57]:
def encoder3(input):
    #encoder
    #input = 28 x 28 x 1 (wide and thin)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input) #28 x 28 x 32
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPool2D(pool_size=(2, 2))(conv1) #14 x 14 x 32
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1) #14 x 14 x 64
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPool2D(pool_size=(2, 2))(conv2) #7 x 7 x 64
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2) #7 x 7 x 128 (small and thick)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)

    return conv3

def decoder3(conv3):    
    #decoder
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv3) #7 x 7 x 64
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    up1 = UpSampling2D((2,2))(conv6) #14 x 14 x 64
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up1) # 14 x 14 x 32
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    up2 = UpSampling2D((2,2))(conv7) # 28 x 28 x 32
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(up2) # 28 x 28 x 1
    return decoded


autoencoder3 = Model(input, decoder3(encoder3(input)))
autoencoder3.compile(loss='mean_squared_error', optimizer = 'rmsprop',metrics =['accuracy'])

In [58]:
autoencoder_train3 = autoencoder3.fit(train_X3, train_ground3, batch_size=32,epochs=5,verbose=1,validation_data=(valid_X3, valid_ground3))

Train on 160 samples, validate on 40 samples
Epoch 1/5
160/160 [==============================] - 4s 26ms/step - loss: 0.1354 - acc: 0.0061 - val_loss: 0.0776 - val_acc: 0.0018
Epoch 2/5
160/160 [==============================] - 2s 13ms/step - loss: 0.0494 - acc: 0.0080 - val_loss: 0.0606 - val_acc: 0.0018
Epoch 3/5
160/160 [==============================] - 2s 14ms/step - loss: 0.0396 - acc: 0.0081 - val_loss: 0.0394 - val_acc: 0.0018
Epoch 4/5
160/160 [==============================] - 2s 13ms/step - loss: 0.0445 - acc: 0.0081 - val_loss: 0.0398 - val_acc: 0.0018
Epoch 5/5
160/160 [==============================] - 2s 13ms/step - loss: 0.0334 - acc: 0.0081 - val_loss: 0.0243 - val_acc: 0.0018


In [59]:
imgs43 = np.reshape(imgs43, [ 200, 28, 28])
imgs1 = np.reshape(imgs43, [ 200, 28, 28])
train_images3, test_images3, train_labels3, test_labels3 = train_test_split(imgs43, d43, test_size=0.33, random_state=42)
print('Training data shape : ', train_images3.shape, len(train_labels3))
print('Testing data shape : ', test_images3.shape, len(test_labels3))
classes = np.unique(train_labels3)
#classes=np.append(classes,0)
nClasses = len(classes)

print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Training data shape :  (134, 28, 28) 134
Testing data shape :  (66, 28, 28) 66
Total number of outputs :  5
Output classes :  ['cat' 'dog' 'hor' 'hum' 'pan']


In [60]:
train_labels3=[0 if x=='hum' else 1 for x in train_labels3]
print(train_labels3)
len(train_labels3)

[1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0]


134

In [61]:
test_labels3=[0 if x=='hum' else 1 for x in test_labels3]
print(test_labels3)
len(test_labels3)

[0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]


66

In [62]:
nRows,nCols = train_images3.shape[1:]
nDims = nRows
print(nCols)
train_data3 = train_images3.reshape(train_images3.shape[0], nRows, nCols, 1)
test_data3 = test_images3.reshape(test_images3.shape[0], nRows, nCols, 1)
input_shape = (nRows, nCols, 1)
train_data3 = train_data3.astype('float32')
test_data3 = test_data3.astype('float32')
train_data3 /= 255
test_data3 /= 255
print(len(train_labels3))
print(len(test_labels3))
train_labels_one_hot3 = to_categorical(train_labels3)
test_labels_one_hot3 = to_categorical(test_labels3)
print('Original label : ', train_labels3[56])
print('After conversion to categorical ( one-hot ) : ', train_labels_one_hot3[56])
print('Original label : ', test_labels3[56])
print('After conversion to categorical ( one-hot ) : ', test_labels_one_hot3[56])

28
134
66
Original label :  0
After conversion to categorical ( one-hot ) :  [1. 0.]
Original label :  0
After conversion to categorical ( one-hot ) :  [1. 0.]


In [63]:
def fc2(enco):
    flat = Flatten()(enco)
    den = Dense(128, activation='relu')(flat)
    out = Dense(2, activation='softmax')(den)
    return out

encode3 = encoder3(input)
full_model3 = Model(input,fc2(encode3))

In [64]:
for l1,l2 in zip(full_model3.layers[:14],autoencoder3.layers[0:14]):
    l1.set_weights(l2.get_weights())

In [65]:
for layer in full_model3.layers[0:14]:
    layer.trainable = False

In [66]:
full_model3.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [67]:
classify_train3 = full_model3.fit(train_data3, train_labels_one_hot3, batch_size=32,epochs=10,verbose=1,validation_data=(test_data3, test_labels_one_hot3))

Train on 134 samples, validate on 66 samples
Epoch 1/10
134/134 [==============================] - 2s 15ms/step - loss: 1.6746 - acc: 0.8209 - val_loss: 3.4190 - val_acc: 0.7879
Epoch 2/10
134/134 [==============================] - 1s 4ms/step - loss: 1.9407 - acc: 0.8731 - val_loss: 3.4190 - val_acc: 0.7879
Epoch 3/10
134/134 [==============================] - 0s 4ms/step - loss: 1.9245 - acc: 0.8806 - val_loss: 3.4190 - val_acc: 0.7879
Epoch 4/10
134/134 [==============================] - 0s 4ms/step - loss: 1.9361 - acc: 0.8731 - val_loss: 3.4190 - val_acc: 0.7879
Epoch 5/10
134/134 [==============================] - 0s 4ms/step - loss: 1.9365 - acc: 0.8731 - val_loss: 3.4190 - val_acc: 0.7879
Epoch 6/10
134/134 [==============================] - 0s 3ms/step - loss: 1.9245 - acc: 0.8806 - val_loss: 3.4190 - val_acc: 0.7879
Epoch 7/10
134/134 [==============================] - 0s 3ms/step - loss: 1.9245 - acc: 0.8806 - val_loss: 3.4190 - val_acc: 0.7879
Epoch 8/10
134/134 [==========

In [68]:
full_model3.evaluate(test_data3, test_labels_one_hot3)

66/66 [==============================] - 0s 2ms/step


[3.4189900521076093, 0.7878787878787878]

In [72]:
filenames = sorted(os.listdir(train_4))
p4=0
d_4=[]
for img_name in filenames:
    img = plt.imread(train_4 + img_name)
    img  = np.resize(img, (28,28))
    if p4==0:
      imgs_4=(img)
      p4=1
    else:
      imgs_4 = np.append(imgs_4, img, axis=0)
    res = img_name[:3]
    d_4.append(res)
print(d_4)
    
print(imgs_4.shape)
        
img_data = np.array(imgs_4)
img_data = img_data.astype('float32')
img_data = img_data/255
img_data.shape

['cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat'

(22400, 28)

In [73]:
imgs_4 = np.reshape(imgs_4, [ 800, 28, 28])
imgs1 = np.reshape(imgs_4, [ 800, 28, 28])
train_images_4, test_images_4, train_labels_4, test_labels_4 = train_test_split(imgs_4, d_4, test_size=0.33, random_state=42)
print('Training data shape : ', train_images_4.shape, len(train_labels_4))
print('Testing data shape : ', test_images_4.shape, len(test_labels_4))
classes = np.unique(train_labels_4)
#classes=np.append(classes,0)
nClasses = len(classes)

print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Training data shape :  (536, 28, 28) 536
Testing data shape :  (264, 28, 28) 264
Total number of outputs :  4
Output classes :  ['cat' 'dog' 'hor' 'hum']


In [74]:
train_labels_4=[0 if x=='cat' else 1 if x == 'dog' else 2 if x =='hum' else 3 for x in train_labels_4]
print(train_labels_4)
len(train_labels_4)

[3, 1, 1, 0, 3, 2, 0, 0, 3, 1, 3, 0, 3, 3, 1, 3, 3, 1, 1, 3, 3, 0, 2, 1, 0, 1, 3, 2, 0, 2, 3, 0, 0, 1, 0, 3, 0, 3, 2, 2, 3, 3, 3, 0, 3, 0, 0, 0, 2, 2, 3, 0, 3, 3, 0, 2, 2, 1, 2, 3, 0, 0, 1, 1, 3, 3, 1, 3, 3, 2, 2, 1, 3, 0, 1, 0, 1, 3, 2, 2, 2, 1, 2, 1, 1, 0, 3, 1, 3, 1, 0, 2, 1, 0, 0, 3, 2, 1, 0, 3, 0, 1, 1, 0, 2, 0, 0, 2, 1, 1, 0, 2, 2, 2, 3, 1, 2, 0, 1, 2, 3, 3, 1, 1, 3, 0, 2, 2, 2, 3, 1, 1, 3, 1, 1, 3, 0, 1, 1, 3, 1, 0, 2, 2, 0, 0, 3, 3, 0, 3, 2, 3, 2, 0, 1, 3, 3, 2, 0, 0, 2, 2, 0, 3, 0, 3, 2, 3, 1, 1, 2, 0, 1, 0, 1, 1, 1, 0, 0, 2, 1, 1, 2, 0, 3, 3, 1, 1, 0, 3, 0, 0, 0, 1, 1, 2, 3, 3, 1, 2, 0, 0, 3, 3, 1, 3, 1, 0, 3, 0, 1, 3, 0, 3, 1, 1, 0, 0, 1, 1, 3, 3, 3, 3, 2, 0, 0, 3, 2, 3, 0, 0, 1, 2, 3, 3, 2, 3, 2, 3, 0, 2, 0, 2, 0, 2, 0, 0, 3, 0, 2, 2, 3, 1, 3, 1, 0, 2, 3, 1, 1, 0, 2, 1, 3, 2, 2, 2, 0, 2, 2, 3, 2, 2, 2, 0, 2, 0, 3, 3, 2, 0, 1, 2, 0, 3, 3, 0, 0, 0, 3, 2, 1, 2, 2, 0, 2, 0, 3, 1, 1, 2, 3, 3, 1, 1, 3, 0, 2, 2, 3, 3, 0, 0, 3, 2, 2, 1, 3, 1, 0, 0, 2, 0, 2, 0, 1, 1, 3, 0, 0, 1, 3, 

536

In [75]:
test_labels_4 =[0 if x=='cat' else 1 if x == 'dog' else 2 if x =='hum' else 3 for x in test_labels_4]
print(test_labels_4)
len(test_labels_4)

[2, 2, 0, 3, 0, 2, 1, 3, 2, 3, 0, 1, 2, 0, 1, 1, 1, 1, 2, 3, 2, 3, 2, 2, 1, 0, 1, 2, 2, 3, 1, 0, 3, 1, 0, 0, 2, 1, 2, 2, 0, 1, 0, 1, 3, 3, 3, 3, 2, 3, 1, 0, 2, 1, 1, 1, 3, 1, 0, 0, 3, 2, 2, 1, 1, 3, 0, 2, 1, 0, 3, 0, 2, 0, 0, 0, 1, 3, 3, 3, 3, 0, 0, 0, 3, 2, 2, 1, 1, 0, 2, 2, 1, 1, 2, 2, 2, 2, 0, 2, 0, 3, 0, 0, 0, 1, 1, 2, 1, 1, 3, 3, 0, 0, 2, 0, 3, 2, 1, 1, 1, 1, 0, 2, 1, 1, 3, 0, 1, 2, 2, 1, 0, 2, 1, 3, 2, 1, 2, 3, 1, 3, 3, 1, 0, 3, 0, 1, 2, 3, 2, 0, 2, 2, 3, 3, 2, 2, 3, 2, 1, 2, 1, 1, 1, 0, 0, 1, 1, 3, 1, 2, 0, 1, 1, 2, 0, 2, 2, 3, 2, 3, 0, 2, 2, 0, 3, 2, 3, 2, 3, 3, 1, 3, 3, 0, 2, 0, 2, 3, 1, 0, 0, 3, 0, 0, 0, 2, 1, 3, 2, 0, 0, 0, 0, 0, 3, 0, 1, 2, 2, 0, 2, 1, 0, 3, 0, 2, 2, 0, 3, 1, 0, 1, 1, 1, 3, 1, 0, 1, 3, 3, 2, 3, 0, 3, 0, 1, 0, 1, 3, 2, 1, 1, 3, 1, 0, 2, 1, 1, 0, 1, 2, 2]


264

In [76]:
nRows,nCols = train_images_4.shape[1:]
nDims = nRows
print(nCols)
train_data_4 = train_images_4.reshape(train_images_4.shape[0], nRows, nCols, 1)
test_data_4 = test_images_4.reshape(test_images_4.shape[0], nRows, nCols, 1)
input_shape = (nRows, nCols, 1)
train_data_4 = train_data_4.astype('float32')
test_data_4 = test_data_4.astype('float32')
train_data_4 /= 255
test_data_4 /= 255
print(len(train_labels_4))
print(len(test_labels_4))
train_labels_one_hot_4 = to_categorical(train_labels_4)
test_labels_one_hot_4 = to_categorical(test_labels_4)
print('Original label : ', train_labels_4[56])
print('After conversion to categorical ( one-hot ) : ', train_labels_one_hot_4[56])
print('Original label : ', test_labels_4[56])
print('After conversion to categorical ( one-hot ) : ', test_labels_one_hot_4[56])

28
536
264
Original label :  2
After conversion to categorical ( one-hot ) :  [0. 0. 1. 0.]
Original label :  3
After conversion to categorical ( one-hot ) :  [0. 0. 0. 1.]


In [77]:
for layer in autoencoder2.layers[0:14]:
    layer.trainable =True
for layer in autoencoder1.layers[0:14]:
    layer.trainable =True
for layer in autoencoder3.layers[0:14]:
    layer.trainable =True
for layer in autoencoder.layers[0:14]:
    layer.trainable =True
    
from keras.layers import *

a = Flatten()(encoder(input))
b = Flatten()(encoder1(input))
c = Flatten()(encoder2(input))
d = Flatten()(encoder3(input))
merged_model = concatenate([a,b,c,d])
#merged_model1 = Dense(256, activation = 'relu')(merged_model1)
merged_model1 = Dense(128, activation = 'relu')(merged_model)
merged_model1 = Dense(4, activation = 'softmax')(merged_model1)
model51 = Model(input,merged_model1)

In [78]:
for layer in autoencoder2.layers[0:14]:
    layer.trainable =False
for layer in autoencoder1.layers[0:14]:
    layer.trainable =False
for layer in autoencoder3.layers[0:14]:
    layer.trainable =False
for layer in autoencoder.layers[0:14]:
    layer.trainable =False

In [79]:
history = model51.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [80]:
history3 = model51.fit(train_data_4,train_labels_one_hot_4,batch_size=32,epochs=20,validation_data=(test_data_4, test_labels_one_hot_4))

Train on 536 samples, validate on 264 samples
Epoch 1/20
536/536 [==============================] - 23s 42ms/step - loss: 3.0880 - acc: 0.7285 - val_loss: 4.6876 - val_acc: 0.6212
Epoch 2/20
536/536 [==============================] - 18s 34ms/step - loss: 2.8907 - acc: 0.7285 - val_loss: 2.4817 - val_acc: 0.7292
Epoch 3/20
536/536 [==============================] - 18s 34ms/step - loss: 2.3625 - acc: 0.7649 - val_loss: 1.9890 - val_acc: 0.7633
Epoch 4/20
536/536 [==============================] - 19s 35ms/step - loss: 2.3459 - acc: 0.7687 - val_loss: 1.9664 - val_acc: 0.7670
Epoch 5/20
536/536 [==============================] - 18s 34ms/step - loss: 2.3789 - acc: 0.7659 - val_loss: 2.1944 - val_acc: 0.7614
Epoch 6/20
536/536 [==============================] - 18s 34ms/step - loss: 2.4117 - acc: 0.7771 - val_loss: 1.9694 - val_acc: 0.7633
Epoch 7/20
536/536 [==============================] - 18s 34ms/step - loss: 2.3354 - acc: 0.7845 - val_loss: 2.2426 - val_acc: 0.7652
Epoch 8/20
536/5

In [82]:
model51.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_70 (Conv2D)              (None, 28, 28, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_76 (Conv2D)              (None, 28, 28, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_82 (Conv2D)              (None, 28, 28, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_88 

In [81]:
filenames = sorted(os.listdir(train5))
p4=0
d44=[]
for img_name in filenames:
    img = plt.imread(train5 + img_name)
    img  = np.resize(img, (28,28))
    if p4==0:
      imgs44=(img)
      p4=1
    else:
      imgs44 = np.append(imgs44, img, axis=0)
    res = img_name[:3]
    d44.append(res)
print(d44)
    
print(imgs44.shape)
        
img_data = np.array(imgs44)
img_data = img_data.astype('float32')
img_data = img_data/255
img_data.shape

['cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'hor', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan', 'pan'

(5600, 28)

In [83]:
imgs444 = imgs44.reshape(-1,28,28,1)
print(imgs444.shape)
imgs444 = imgs444 / np.max(imgs444)

(200, 28, 28, 1)


In [84]:
train_X4,valid_X4,train_ground4,valid_ground4 = train_test_split(imgs444,imgs444,test_size=0.2,random_state=13)

In [85]:
def encoder4(input):
    #encoder
    #input = 28 x 28 x 1 (wide and thin)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input) #28 x 28 x 32
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPool2D(pool_size=(2, 2))(conv1) #14 x 14 x 32
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1) #14 x 14 x 64
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPool2D(pool_size=(2, 2))(conv2) #7 x 7 x 64
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2) #7 x 7 x 128 (small and thick)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)

    return conv3

def decoder4(conv3):    
    #decoder
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv3) #7 x 7 x 64
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    up1 = UpSampling2D((2,2))(conv6) #14 x 14 x 64
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up1) # 14 x 14 x 32
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    up2 = UpSampling2D((2,2))(conv7) # 28 x 28 x 32
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(up2) # 28 x 28 x 1
    return decoded


autoencoder4 = Model(input, decoder4(encoder4(input)))
autoencoder4.compile(loss='mean_squared_error', optimizer = 'rmsprop',metrics =['accuracy'])

In [86]:
autoencoder_train4 = autoencoder4.fit(train_X4, train_ground4, batch_size=32,epochs=5,verbose=1,validation_data=(valid_X4, valid_ground4))

Train on 160 samples, validate on 40 samples
Epoch 1/5
160/160 [==============================] - 6s 35ms/step - loss: 0.0833 - acc: 0.0254 - val_loss: 0.0465 - val_acc: 0.0201
Epoch 2/5
160/160 [==============================] - 3s 16ms/step - loss: 0.0287 - acc: 0.0317 - val_loss: 0.0234 - val_acc: 0.0201
Epoch 3/5
160/160 [==============================] - 2s 13ms/step - loss: 0.0245 - acc: 0.0314 - val_loss: 0.0198 - val_acc: 0.0201
Epoch 4/5
160/160 [==============================] - 2s 13ms/step - loss: 0.0206 - acc: 0.0317 - val_loss: 0.0272 - val_acc: 0.0202
Epoch 5/5
160/160 [==============================] - 2s 13ms/step - loss: 0.0187 - acc: 0.0318 - val_loss: 0.0167 - val_acc: 0.0202


In [87]:
imgs44 = np.reshape(imgs44, [ 200, 28, 28])
imgs1 = np.reshape(imgs44, [ 200, 28, 28])
train_images4, test_images4, train_labels4, test_labels4 = train_test_split(imgs44, d44, test_size=0.33, random_state=42)
print('Training data shape : ', train_images4.shape, len(train_labels4))
print('Testing data shape : ', test_images4.shape, len(test_labels4))
classes = np.unique(train_labels4)
#classes=np.append(classes,0)
nClasses = len(classes)

print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Training data shape :  (134, 28, 28) 134
Testing data shape :  (66, 28, 28) 66
Total number of outputs :  4
Output classes :  ['cat' 'dog' 'hor' 'pan']


In [88]:
train_labels4=[0 if x=='pan' else 1 for x in train_labels4]
print(train_labels4)
len(train_labels4)

[1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0]


134

In [89]:
test_labels4=[0 if x=='pan' else 1 for x in test_labels4]
print(test_labels4)
len(test_labels4)

[1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0]


66

In [90]:
nRows,nCols = train_images4.shape[1:]
nDims = nRows
print(nCols)
train_data4 = train_images4.reshape(train_images4.shape[0], nRows, nCols, 1)
test_data4 = test_images4.reshape(test_images4.shape[0], nRows, nCols, 1)
input_shape = (nRows, nCols, 1)
train_data4 = train_data4.astype('float32')
test_data4 = test_data4.astype('float32')
train_data4 /= 255
test_data4 /= 255
print(len(train_labels4))
print(len(test_labels4))
train_labels_one_hot4 = to_categorical(train_labels4)
test_labels_one_hot4 = to_categorical(test_labels4)
print('Original label : ', train_labels4[56])
print('After conversion to categorical ( one-hot ) : ', train_labels_one_hot4[56])
print('Original label : ', test_labels4[56])
print('After conversion to categorical ( one-hot ) : ', test_labels_one_hot4[56])

28
134
66
Original label :  0
After conversion to categorical ( one-hot ) :  [1. 0.]
Original label :  0
After conversion to categorical ( one-hot ) :  [1. 0.]


In [91]:
def fc3(enco):
    flat = Flatten()(enco)
    den = Dense(128, activation='relu')(flat)
    out = Dense(2, activation='softmax')(den)
    return out

encode4 = encoder4(input)
full_model4 = Model(input,fc3(encode4))

In [92]:
for l1,l2 in zip(full_model4.layers[:14],autoencoder4.layers[0:14]):
    l1.set_weights(l2.get_weights())

In [93]:
for layer in full_model4.layers[0:14]:
    layer.trainable = False

In [94]:
full_model4.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [95]:
classify_train4 = full_model4.fit(train_data4,train_labels_one_hot4,batch_size=32,epochs=5,validation_data=(test_data4, test_labels_one_hot4))

Train on 134 samples, validate on 66 samples
Epoch 1/5
134/134 [==============================] - 3s 23ms/step - loss: 1.9662 - acc: 0.5896 - val_loss: 3.4354 - val_acc: 0.6212
Epoch 2/5
134/134 [==============================] - 0s 4ms/step - loss: 1.7219 - acc: 0.7239 - val_loss: 2.5532 - val_acc: 0.5909
Epoch 3/5
134/134 [==============================] - 0s 4ms/step - loss: 1.2140 - acc: 0.7313 - val_loss: 1.7404 - val_acc: 0.6212
Epoch 4/5
134/134 [==============================] - 1s 4ms/step - loss: 0.8671 - acc: 0.7985 - val_loss: 2.0834 - val_acc: 0.5455
Epoch 5/5
134/134 [==============================] - 0s 4ms/step - loss: 1.0708 - acc: 0.7612 - val_loss: 1.9439 - val_acc: 0.7121


In [97]:
filenames = sorted(os.listdir(train_5))
p4=0
d_5=[]
for img_name in filenames:
    img = plt.imread(train_5 + img_name)
    img  = np.resize(img, (28,28))
    if p4==0:
      imgs_5=(img)
      p4=1
    else:
      imgs_5 = np.append(imgs_5, img, axis=0)
    res = img_name[:3]
    d_5.append(res)
print(d_5)
    
print(imgs_5.shape)
        
img_data = np.array(imgs_5)
img_data = img_data.astype('float32')
img_data = img_data/255
img_data.shape

['cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat'

(28000, 28)

In [98]:
imgs_5 = np.reshape(imgs_5, [ 1000, 28, 28])
imgs1 = np.reshape(imgs_5, [ 1000, 28, 28])
train_images_5, test_images_5, train_labels_5, test_labels_5 = train_test_split(imgs_5, d_5, test_size=0.33, random_state=42)
print('Training data shape : ', train_images_5.shape, len(train_labels_5))
print('Testing data shape : ', test_images_5.shape, len(test_labels_5))
classes = np.unique(train_labels_5)
#classes=np.append(classes,0)
nClasses = len(classes)

print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Training data shape :  (670, 28, 28) 670
Testing data shape :  (330, 28, 28) 330
Total number of outputs :  5
Output classes :  ['cat' 'dog' 'hor' 'hum' 'pan']


In [99]:
train_labels_5=[0 if x=='cat' else 1 if x == 'dog' else 2 if x =='hum' else 3 if x =='hor' else 4 for x in train_labels_5]
print(train_labels_5)
len(train_labels_5)

[2, 1, 2, 2, 0, 1, 2, 3, 4, 3, 3, 3, 0, 0, 1, 3, 3, 4, 0, 4, 0, 1, 4, 0, 1, 0, 1, 3, 1, 0, 3, 3, 0, 3, 0, 0, 2, 0, 1, 4, 0, 2, 0, 0, 0, 2, 3, 0, 4, 4, 1, 0, 2, 1, 1, 4, 3, 0, 4, 2, 0, 0, 2, 0, 3, 0, 3, 3, 2, 4, 0, 3, 3, 3, 2, 2, 4, 1, 0, 3, 3, 4, 2, 1, 4, 2, 2, 4, 3, 3, 0, 2, 3, 3, 3, 1, 0, 0, 2, 1, 1, 1, 1, 3, 1, 0, 0, 1, 4, 2, 1, 3, 4, 2, 3, 3, 2, 0, 2, 2, 3, 4, 1, 0, 1, 2, 4, 2, 1, 3, 0, 0, 3, 4, 1, 0, 0, 3, 3, 3, 3, 0, 4, 0, 1, 4, 4, 4, 2, 3, 2, 0, 0, 1, 3, 0, 0, 1, 0, 3, 4, 3, 2, 0, 3, 0, 3, 3, 4, 4, 2, 0, 1, 4, 2, 4, 2, 1, 4, 2, 1, 3, 1, 0, 1, 2, 1, 0, 4, 0, 1, 2, 4, 1, 3, 3, 4, 1, 2, 0, 3, 0, 2, 1, 4, 3, 0, 0, 2, 3, 0, 1, 1, 3, 0, 4, 3, 4, 3, 0, 0, 3, 2, 0, 4, 4, 1, 1, 0, 1, 0, 4, 4, 2, 1, 3, 2, 1, 1, 4, 0, 0, 2, 0, 1, 4, 0, 3, 1, 2, 1, 0, 4, 2, 4, 3, 3, 2, 0, 0, 1, 4, 2, 2, 4, 0, 1, 2, 3, 2, 0, 0, 1, 1, 2, 4, 0, 0, 2, 2, 0, 1, 2, 2, 2, 2, 4, 3, 4, 0, 4, 0, 4, 0, 0, 3, 0, 4, 4, 3, 4, 3, 1, 2, 4, 1, 0, 4, 3, 4, 1, 1, 0, 4, 1, 3, 4, 4, 2, 4, 0, 4, 4, 3, 4, 4, 3, 4, 0, 4, 0, 4, 4, 

670

In [100]:
test_labels_5=[0 if x=='cat' else 1 if x == 'dog' else 2 if x =='hum' else 3 if x =='hor' else 4 for x in test_labels_5]
print(test_labels_5)
len(test_labels_5)

[3, 2, 2, 2, 3, 2, 2, 3, 4, 0, 4, 0, 2, 4, 4, 4, 1, 4, 2, 1, 3, 0, 1, 3, 1, 1, 0, 4, 4, 4, 1, 0, 2, 3, 1, 0, 2, 3, 4, 0, 1, 0, 1, 4, 1, 1, 4, 2, 3, 4, 3, 4, 1, 3, 0, 0, 2, 0, 1, 4, 1, 0, 3, 1, 2, 2, 2, 4, 0, 3, 0, 3, 3, 2, 1, 1, 2, 0, 3, 1, 2, 0, 1, 0, 1, 3, 4, 2, 2, 1, 1, 3, 3, 1, 2, 3, 2, 0, 0, 4, 2, 1, 4, 1, 2, 2, 3, 4, 0, 4, 3, 4, 4, 4, 0, 3, 0, 4, 3, 1, 4, 3, 2, 1, 1, 3, 3, 3, 4, 4, 3, 4, 1, 4, 3, 0, 0, 1, 0, 0, 0, 1, 2, 1, 4, 4, 4, 1, 4, 1, 2, 0, 4, 3, 4, 3, 1, 2, 0, 4, 3, 2, 2, 2, 4, 2, 1, 2, 1, 1, 1, 3, 0, 4, 3, 1, 1, 1, 4, 2, 3, 2, 1, 3, 4, 0, 1, 1, 4, 2, 4, 4, 2, 0, 1, 3, 1, 1, 2, 0, 0, 3, 2, 3, 4, 3, 0, 4, 3, 4, 0, 0, 4, 4, 2, 1, 2, 1, 4, 2, 1, 2, 4, 0, 4, 0, 3, 1, 4, 3, 1, 4, 3, 3, 0, 0, 1, 4, 0, 1, 0, 1, 3, 2, 1, 0, 3, 0, 0, 1, 0, 4, 2, 4, 1, 1, 3, 4, 4, 0, 2, 3, 0, 1, 1, 4, 2, 1, 2, 1, 3, 0, 4, 1, 3, 0, 0, 2, 0, 2, 3, 2, 1, 1, 1, 2, 1, 3, 1, 4, 4, 2, 3, 3, 1, 3, 4, 3, 0, 0, 3, 3, 3, 3, 2, 4, 1, 3, 4, 3, 3, 3, 3, 2, 3, 3, 1, 1, 2, 1, 0, 0, 1, 2, 2, 3, 1, 3, 2, 0]


330

In [101]:
nRows,nCols = train_images_5.shape[1:]
nDims = nRows
print(nCols)
train_data_5 = train_images_5.reshape(train_images_5.shape[0], nRows, nCols, 1)
test_data_5 = test_images_5.reshape(test_images_5.shape[0], nRows, nCols, 1)
input_shape = (nRows, nCols, 1)
train_data_5 = train_data_5.astype('float32')
test_data_5 = test_data_5.astype('float32')
train_data_5 /= 255
test_data_5 /= 255
print(len(train_labels_5))
print(len(test_labels_5))
train_labels_one_hot_5 = to_categorical(train_labels_5)
test_labels_one_hot_5 = to_categorical(test_labels_5)
print('Original label : ', train_labels_5[56])
print('After conversion to categorical ( one-hot ) : ', train_labels_one_hot_5[56])
print('Original label : ', test_labels_5[56])
print('After conversion to categorical ( one-hot ) : ', test_labels_one_hot_5[56])

28
670
330
Original label :  3
After conversion to categorical ( one-hot ) :  [0. 0. 0. 1. 0.]
Original label :  2
After conversion to categorical ( one-hot ) :  [0. 0. 1. 0. 0.]


In [155]:
for layer in model51.layers[0:62]:
    layer.trainable =True
for layer in autoencoder4.layers[0:14]:
    layer.trainable =True
    
from keras.layers import *

a = Flatten()(encoder(input))
b = Flatten()(encoder1(input))
c = Flatten()(encoder2(input))
d = Flatten()(encoder3(input))
e = Flatten()(encoder4(input))
merged_model2 = concatenate([merged_model,e])
#merged_model1 = Dense(256, activation = 'relu')(merged_model1)
merged_model2 = Dense(128, activation = 'relu')(merged_model2)
merged_model2 = Dense(5, activation = 'softmax')(merged_model2)
model151 = Model(input,merged_model2)

In [156]:
for layer in model51.layers[0:62]:
    layer.trainable =False
for layer in autoencoder4.layers[0:14]:
    layer.trainable =False

In [157]:
model151.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_70 (Conv2D)              (None, 28, 28, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_76 (Conv2D)              (None, 28, 28, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_82 (Conv2D)              (None, 28, 28, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_88 

In [158]:
history1 = model151.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [159]:
history1 = model151.fit(train_data_5,train_labels_one_hot_5,batch_size=32,epochs=20,validation_data=(test_data_5, test_labels_one_hot_5))

Train on 670 samples, validate on 330 samples
Epoch 1/20
670/670 [==============================] - 23s 35ms/step - loss: 3.2323 - acc: 0.7749 - val_loss: 3.4988 - val_acc: 0.7794
Epoch 2/20
670/670 [==============================] - 14s 21ms/step - loss: 3.2737 - acc: 0.7881 - val_loss: 3.6291 - val_acc: 0.7721
Epoch 3/20
670/670 [==============================] - 14s 21ms/step - loss: 3.3121 - acc: 0.7845 - val_loss: 3.5973 - val_acc: 0.7685
Epoch 4/20
670/670 [==============================] - 14s 22ms/step - loss: 3.4375 - acc: 0.7821 - val_loss: 3.5957 - val_acc: 0.7733
Epoch 5/20
670/670 [==============================] - 15s 22ms/step - loss: 3.3628 - acc: 0.7869 - val_loss: 3.6471 - val_acc: 0.7685
Epoch 6/20
670/670 [==============================] - 14s 21ms/step - loss: 3.3510 - acc: 0.7875 - val_loss: 3.8489 - val_acc: 0.7539
Epoch 7/20
670/670 [==============================] - 14s 21ms/step - loss: 3.5412 - acc: 0.7785 - val_loss: 4.0967 - val_acc: 0.7418
Epoch 8/20
670/6

In [160]:
for layer in model151.layers[0:76]:
    layer.trainable =True

In [161]:
history1 = model151.fit(train_data_5,train_labels_one_hot_5,batch_size=32,epochs=20,validation_data=(test_data_5, test_labels_one_hot_5))

C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Train on 670 samples, validate on 330 samples
Epoch 1/20
670/670 [==============================] - 15s 22ms/step - loss: 3.4592 - acc: 0.7821 - val_loss: 3.3640 - val_acc: 0.7624
Epoch 2/20
670/670 [==============================] - 14s 21ms/step - loss: 3.4053 - acc: 0.7857 - val_loss: 3.6464 - val_acc: 0.7697
Epoch 3/20
670/670 [==============================] - 15s 22ms/step - loss: 3.4445 - acc: 0.7839 - val_loss: 3.4841 - val_acc: 0.7818
Epoch 4/20
670/670 [==============================] - 16s 24ms/step - loss: 3.3971 - acc: 0.7857 - val_loss: 3.4479 - val_acc: 0.7842
Epoch 5/20
670/670 [==============================] - 15s 23ms/step - loss: 3.3679 - acc: 0.7893 - val_loss: 3.4300 - val_acc: 0.7842
Epoch 6/20
670/670 [==============================] - 14s 21ms/step - loss: 3.4319 - acc: 0.7839 - val_loss: 3.6728 - val_acc: 0.7709
Epoch 7/20
670/670 [==============================] - 14s 21ms/step - loss: 3.4656 - acc: 0.7827 - val_loss: 3.6777 - val_acc: 0.7685
Epoch 8/20
670/6